## STEP 1

In [32]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [9]:
# load the dataset
df = pd.read_csv('./learn-ai-bbc/BBC News Train.csv')

# remove unnecessary columns
df = df.drop(columns=["ArticleId"])


In [10]:
def clean_text(text):
    # remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # convert to lowercase
    text = text.lower()
    
    # tokenize the text
    words = word_tokenize(text)
    
    # remove stop words
    words = [word for word in words if word not in stopwords.words("english")]
    
    # stem the words
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # join the words back into a string
    text = " ".join(words)
    
    return text

# apply the clean_text function to the Text column
df["Text"] = df["Text"].apply(clean_text)

# get the unique categories


In [12]:
categories = df["Category"].unique()

# define a function to calculate the TF-ICF score
def tf_icf(term, category):
    # calculate term frequency
    tf = df[df["Category"] == category]["Text"].str.count(term).sum()
    
    # calculate class frequency
    cf = df["Text"].str.contains(term).sum()
    
    # calculate inverse-class frequency
    icf = np.log10(len(categories) / cf)
    
    # calculate TF-ICF score
    tf_icf_score = tf * icf
    
    return tf_icf_score

# create a TF-IDF vectorizer object with the TF-ICF weighting scheme
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words("english"), norm=None, use_idf=False)

# calculate the TF-ICF scores for each term in each category
tf_icf_scores = {}
for category in categories:
    # get the documents in the current category
    docs = df[df["Category"] == category]["Text"]
    
    # fit the TF-IDF vectorizer on the documents
    tfidf_vectorizer.fit(docs)
    
    # get the vocabulary and term frequencies from the vectorizer
    vocabulary = tfidf_vectorizer.vocabulary_
    term_freqs = tfidf_vectorizer.transform(docs).sum(axis=0)
    
    # calculate the TF-ICF scores for each term in the vocabulary
    for term, index in vocabulary.items():
        tf_icf_score = tf_icf(term, category)
        if term not in tf_icf_scores:
            tf_icf_scores[term] = np.zeros(len(categories))
        tf_icf_scores[term][np.where(categories == category)[0][0]] = tf_icf_score
    
# transform the documents into TF-ICF vectors
tf_icf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize, stop_words=stopwords.words("english"), vocabulary=tfidf_vectorizer.vocabulary_, use_idf=False, norm="l2")
tf_icf_vectors = tf_icf_vectorizer.fit_transform(df["Text"])

C:\Users\anike\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
C:\Users\anike\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


## STEP 2

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(tf_icf_vectors, list(df['Category']), test_size = 0.3, random_state = 42)

## STEP 3, 4

In [33]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, Y_train)
Y_pred = list(clf.predict(X_test))
from sklearn.metrics import classification_report

print(classification_report(Y_test, Y_pred))

               precision    recall  f1-score   support

     business       0.92      0.98      0.95       108
entertainment       1.00      0.84      0.91        79
     politics       0.93      0.93      0.93        86
        sport       0.94      1.00      0.97       101
         tech       0.93      0.92      0.92        73

     accuracy                           0.94       447
    macro avg       0.94      0.93      0.94       447
 weighted avg       0.94      0.94      0.94       447

